# Modify NumPy arrays in BigDataViewer

Note that this notebook does not run on OSX! You can copy the contents of the relevant cells into a file and run the file through thw `OSXAWTwrapper`:
```bash
WRAPPER="$(python -c 'import os; import site; print( os.path.join(site.getsitepackages()[0], "imglyb", "OSXAWTwrapper.py") )')"
python $WRAPPER /path/to/file
```

In [1]:
import math
import numpy as np

import imglyb
import imglyb.util as util

from jnius import autoclass, PythonJavaClass, java_method, cast

In [2]:
RealPoint = autoclass( 'net.imglib2.RealPoint' )

class Painter( PythonJavaClass ):
    __javainterfaces__ = ['org/scijava/ui/behaviour/DragBehaviour']


    def __init__( self, img, mask, viewer, paint_listener = lambda : None ):
        super( Painter, self ).__init__()
        self.img = img
        self.mask = mask
        self.radius = int( mask.shape[0] / 2 )
        self.viewer = viewer
        self.oX = 0
        self.oY = 0
        self.n_dim = len( img.shape )
        self.labelLocation = RealPoint( 3 )
        self.lower = np.empty( ( self.n_dim, ), dtype=np.int32 )
        self.upper = np.empty( ( self.n_dim, ), dtype=np.int32 )
        self.paint_listener = paint_listener

    @java_method('(II)V')
    def init( self, x, y ):
        self._paint( x, y )
        self.oX = x
        self.oY = y
        self.viewer.requestRepaint()

    @java_method('(II)V')
    def drag( self, x, y ):
        self._setCoordinates( self.oX, self.oY )
        n_dim = self.labelLocation.numDimensions()
        origin = np.array( [ self.labelLocation.getDoublePosition( d ) for d in range( n_dim ) ] )
        origin_p = RealPoint( n_dim )
        for d, p in enumerate( origin ):
            origin_p.setPosition( p, d )
        self._setCoordinates( x, y )
        target = np.array( [ self.labelLocation.getDoublePosition( d ) for d in range( n_dim ) ] )
        diff = target - origin
        length = np.linalg.norm( diff )
        direction = diff / length
        try:
            for l in range( 1, math.ceil( length ) ):
                for d, dist in enumerate( direction ):
                    origin_p.move( dist, d )
                self._paint_at_localizable( origin_p )
        except Exception as e:
            print( e )
            raise e

        self.oX = x
        self.oY = y
        self.viewer.requestRepaint()

    @java_method('(II)V')
    def end( self, x, y ):
        self.paint_listener()

    def _paint( self, x, y ):
        self._setCoordinates( x, y )
        self._paint_at_localizable( self.labelLocation )

    def _paint_at_localizable( self, labelLocation ):
        for d in range( self.n_dim ):
            int_pos = int( round( labelLocation.getDoublePosition( d ) ) )
            if int_pos < 0 or int_pos >= self.img.shape[ ::-1 ][ d ]:
                return
            self.lower[ d ] = int_pos - self.radius
            self.upper[ d ] = int_pos + self.radius
            
        self.lower = self.lower[::-1]
        self.upper = self.upper[::-1]

        img_lower = np.maximum( self.lower, 0 )
        img_upper = np.minimum( self.upper, self.img.shape )
        
        if np.any( img_lower >= img_upper ):
            return

        mask_lower = np.abs( np.minimum( self.lower, 0 ) ).astype( self.lower.dtype )
        mask_upper = np.minimum( mask_lower + ( img_upper - img_lower ), mask_lower + np.array( self.mask.shape ) )
        img_selection = tuple( slice(l, u) for l, u in zip( img_lower, img_upper ) )
        mask_selection = tuple( slice(l, u) for l, u in zip( mask_lower, mask_upper ) )

        # color_channels = tuple( 255 << ( ( idx * 8 ) if np.random.rand() > 0.5 else 0 ) for idx in range( self.n_dim )  )
        color_channels = tuple( np.random.randint( 128, 255 ) << ( idx * 8 ) for idx in range( self.n_dim )  )
        color = 0
        for c in color_channels:
            color = color | c
        try:
            self.img[ img_selection  ][ self.mask[ mask_selection ] ] = color
        except Exception as e:
            print( "EXCEPTION", e )
            raise e

    def _setCoordinates( self, x, y ):
        self.labelLocation.setPosition( x, 0 )
        self.labelLocation.setPosition( y, 1 )
        self.labelLocation.setPosition( 0, 2 )
        self.viewer.displayToGlobalCoordinates( self.labelLocation )

initial_img = np.zeros( ( 300, 200, 100 ), dtype=np.uint32 ) + ( 80 | ( 80 << 8 ) )
initial_rai = imglyb.to_imglib_argb( initial_img )
bdv = util.BdvFunctions.show( initial_rai, 'canvas' )
mask = np.ones( ( 10, 10, 10 ) ) == 1
painter = Painter( initial_img, mask, bdv.getBdvHandle().getViewerPanel() )

print( initial_img.shape, [ initial_rai.dimension( d ) for d in range( initial_rai.numDimensions() ) ] )

behaviors = util.Helpers.behaviours()
behaviors.install( bdv.getBdvHandle().getTriggerbindings(), "paint" )
behaviors.behaviour( painter, "paint", "SPACE button1" )


(300, 200, 100) [100, 200, 300]


/home/hanslovskyp/miniconda3/envs/scipy-2019-examples/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in true_divide
